In [11]:
from __future__ import print_function
import os
import sys
import cv2  # OpenCV用于图像读取和旋转
import numpy as np
from keras.models import load_model

sys.path.append('..')
from utils import load_and_pair_data, manual_train_test_split, angle_error_regression

In [ ]:
model_location = os.path.join('..', 'models', 'rotnet_carcass_regression.hdf5')
model = load_model(model_location, custom_objects={'angle_error_regression': angle_error_regression})
model.summary()

# 设置路径，加载图片数据
image_dir = r"C:\Users\xuyan\RotNet\data\RotationAngle\DATASET\RAW\images"
csv_path = r"C:\Users\xuyan\RotNet\data\RotationAngle\DATASET\dataset_image_rotation_data.csv"

images, angles,origin_images = load_and_pair_data(image_dir, csv_path,target_size=(96, 128))

X_train, X_test, y_train, y_test = manual_train_test_split(images, angles, test_size=0.2)

In [ ]:
batch_size = 8
out = model.evaluate(
    X_test,y_test,
    steps=len(y_test) / batch_size
)

print('Test loss:', out)

In [ ]:
y_pred = model.predict(X_test)
# 打印真实标签和预测标签
print("真实标签 (y_true):")
print(y_test)

In [ ]:
print("预测标签 (y_pred):")
for value in y_pred.flatten():
    print(f"{value:.1f}")

In [ ]:
import matplotlib.pyplot as plt

# Predict labels
y_pred = model.predict(X_test)

# Plot the difference between true labels and predicted labels using line plots
plt.figure(figsize=(10, 6))

# Plot the true labels as a line
plt.plot(y_test, label="True Labels (y_true)", color='b', linestyle='-', marker='o')

# Plot the predicted labels as a line
plt.plot(y_pred, label="Predicted Labels (y_pred)", color='r', linestyle='-', marker='x')

# Add title and labels to the plot
plt.title("Difference Between True Labels and Predicted Labels")
plt.xlabel("Sample Index")
plt.ylabel("Angle (Degrees)")
plt.legend()

# Display the plot
plt.show()



In [ ]:
# 加载数据
images, angles, origin_images = load_and_pair_data(image_dir, csv_path, target_size=(96, 128))

# 将数据格式化为float32
images = images.astype('float32')

# 对所有图片进行预测
y_pred = model.predict(images)

# 输出保存旋转后的图片的路径
output_dir = r"C:\Users\xuyan\RotNet\data\RotationAngle\DATASET\STANDARDIZED\images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 获取图片文件名列表（假设load_and_pair_data返回了原始文件名）
image_filenames = sorted(os.listdir(image_dir))

# 处理图片，旋转校正所有图片并保存
for idx, (image, pred_angle, filename) in enumerate(zip(origin_images, y_pred.flatten(), image_filenames)):
    # 获取图像的原始文件路径，用于保存
    output_path = os.path.join(output_dir, filename)
    
    # 旋转角度为负的预测值
    corrected_angle = -pred_angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # 计算旋转矩阵
    rotation_matrix = cv2.getRotationMatrix2D(center, corrected_angle, 1.0)
    
    # 扩展图像边界，以防旋转时图像内容被裁剪
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    
    # 计算新的边界尺寸，保证旋转后图像不会被裁剪
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # 调整旋转矩阵以考虑平移
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    # 对图像进行仿射变换，并调整边界
    expanded_image = cv2.warpAffine(image, rotation_matrix, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # 将图像裁剪或调整回原始尺寸，以确保最终输出图像和原始输入尺寸一致
    corrected_image = cv2.resize(expanded_image, (w, h), interpolation=cv2.INTER_AREA)
    
    # 保存旋转后的图片，保持原来的文件名
    cv2.imwrite(output_path, corrected_image)

# 显示前十张图片作为参考
for i in range(50):
    # 加载原始图片
    original_image_path = os.path.join(image_dir, image_filenames[i])
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    
    # 加载保存的校正图片
    corrected_image_path = os.path.join(output_dir, image_filenames[i])
    corrected_image = cv2.imread(corrected_image_path, cv2.IMREAD_GRAYSCALE)
    
    # 创建两个子图，显示原图和校正后的图像
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(f"Original and Corrected Image: {image_filenames[i]}", fontsize=16)

    # 显示原图
    axes[0].imshow(original_image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    # 显示校正后的图
    axes[1].imshow(corrected_image, cmap='gray')
    axes[1].set_title("Corrected Image")
    axes[1].axis('off')

    # 显示图像
    plt.show()